# Cleaning Starbucks Rewards JSON Dataset

In [1]:
import pandas as pd
import numpy as np

## Read files and print first three rows

In [2]:
df_portfolio = pd.read_json('./portfolio.json', orient='records', lines=True)
df_profile = pd.read_json('./profile.json', orient='records', lines=True)
df_transcript = pd.read_json('./transcript.json', orient='records', lines=True)

In [3]:
print(f'df_portfolio.shape={df_portfolio.shape}')
display(df_portfolio.head(3))

print(f'df_profile.shape={df_profile.shape}')
display(df_profile.head(3))

print(f'df_transcript.shape={df_transcript.shape}')
display(df_transcript.head(3))

df_portfolio.shape=(10, 6)


,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed


df_profile.shape=(17000, 5)


,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN


df_transcript.shape=(306534, 4)


,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0


## Preprocess DataFrames for easier use

### Portfolio

In [4]:
df_portfolio_processed = df_portfolio.copy()

Rename `id` to `offer_id`.

In [5]:
df_portfolio_processed.rename(columns={
    'id': 'offer_id'
}, inplace=True)

df_portfolio_processed.head(3)

,reward,channels,difficulty,duration,offer_type,offer_id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed


Since the `channels` bcolumn contains a list of strings, create one-hot encoded columns for each channel type.

In [6]:
df_portfolio_processed.head(3)

,reward,channels,difficulty,duration,offer_type,offer_id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed


In [7]:
channels_exploded = df_portfolio_processed['channels'].explode()
df_portfolio_processed = df_portfolio_processed.join(pd.crosstab(channels_exploded.index, channels_exploded))
df_portfolio_processed.drop(columns=['channels'], inplace=True)

df_portfolio_processed.head(3)

,reward,difficulty,duration,offer_type,offer_id,email,mobile,social,web
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,0,1


### Profile

In [8]:
df_profile_processed = df_profile.copy()

In [9]:
df_profile_processed['became_member_on'] = pd.to_datetime(df_profile_processed['became_member_on'], format='%Y%m%d')
df_profile_processed.rename(columns={
    'id': 'member_id'
}, inplace=True)

df_profile_processed.head(3)

,gender,age,member_id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN


### Transcript

In [10]:
df_transcript_processed = df_transcript.copy()

In [11]:
def get_offer_id(value_dict):
    return value_dict['offer id'] if 'offer id' in value_dict else np.nan
    
def get_amount(value_dict):
    return value_dict['amount'] if 'amount' in value_dict else np.nan

In [12]:
df_transcript_processed['offer_id'] = df_transcript_processed['value'].apply(lambda x: get_offer_id(x))
df_transcript_processed['amount'] = df_transcript_processed['value'].apply(lambda x: get_amount(x))

In [13]:
df_transcript_processed.drop(columns=['value'], inplace=True)
df_transcript_processed.rename(columns={
    'person': 'member_id'
}, inplace=True)

In [14]:
df_transcript_processed

,member_id,event,time,offer_id,amount
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN
...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,714,NaN,1.59
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,NaN,9.53
306531,a00058cf10334a308c68e7631c529907,transaction,714,NaN,3.61
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,NaN,3.53


## Save to CSV files

In [15]:
df_portfolio_processed.to_csv('portfolio.csv', index=None)
df_profile_processed.to_csv('profile.csv', index=None)
df_transcript_processed.to_csv('transcript.csv', index=None)